In [1]:
from huggingface_hub import notebook_login
notebook_login()

# from huggingface_hub import login
# login()


In [2]:
CACHE_PATH = '/run/media/srujan/Data/Development/ML-algo-practice/.cache'

In [3]:
from transformers import AutoTokenizer, BitsAndBytesConfig, Gemma3ForCausalLM
import torch

model_id = "google/gemma-3-1b-it"

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model = Gemma3ForCausalLM.from_pretrained(
    model_id, quantization_config=quantization_config,cache_dir = CACHE_PATH
).eval()


# model = Gemma3ForCausalLM.from_pretrained(
#     model_id,cache_dir = CACHE_PATH,
# ).eval()

tokenizer = AutoTokenizer.from_pretrained(model_id,trust_remote_code=True,cache_dir = CACHE_PATH)


Loading weights:   0%|          | 0/340 [00:00<?, ?it/s]

In [4]:

# messages = [
#     [
#         {
#             "role": "system",
#             "content": [{"type": "text", "text": "You are a helpful assistant."}]
#         },
#         {
#             "role": "user",
#             "content": [{"type": "text", "text": "Write a poem on Hugging Face, the company"},]
#         },{
#             "role": "assistant",
#             "content": [{"type": "text", "text": "I dont know"},]
#         },{
#             "role": "user",
#             "content": [{"type": "text", "text": "Why not? give me a reason on why you cant do a poem"},]
#         },
#     ],
# ]


# inputs = tokenizer.apply_chat_template(
#     messages,
#     add_generation_prompt=True,
#     tokenize=True,
#     return_dict=True,
#     return_tensors="pt",
# ).to(model.device).to(torch.bfloat16)


# with torch.inference_mode():
#     outputs = model.generate(**inputs, max_new_tokens=64)

# outputs = tokenizer.batch_decode(outputs)
# print(outputs[0])

In [5]:
from datasets import load_dataset
# ds = load_dataset("nvidia/Nemotron-Post-Training-Dataset-v2",  split=["code", "math"] , cache_dir=CACHE_PATH)
# data = load_dataset("Abirate/english_quotes",cache_dir=CACHE_PATH)
dataset = load_dataset("dvgodoy/yoda_sentences", split="train",cache_dir=CACHE_PATH)
dataset

Dataset({
    features: ['sentence', 'translation', 'translation_extra'],
    num_rows: 720
})

In [6]:
dataset[123]

{'sentence': 'The cup cracked and spilled its contents.',
 'translation': 'Cracked and spilled its contents, the cup did.',
 'translation_extra': 'Cracked and spilled its contents, the cup did. Hrmmm.'}

In [7]:
model

Gemma3ForCausalLM(
  (model): Gemma3TextModel(
    (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 1152, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma3DecoderLayer(
        (self_attn): Gemma3Attention(
          (q_proj): Linear8bitLt(in_features=1152, out_features=1024, bias=False)
          (k_proj): Linear8bitLt(in_features=1152, out_features=256, bias=False)
          (v_proj): Linear8bitLt(in_features=1152, out_features=256, bias=False)
          (o_proj): Linear8bitLt(in_features=1024, out_features=1152, bias=False)
          (q_norm): Gemma3RMSNorm((256,), eps=1e-06)
          (k_norm): Gemma3RMSNorm((256,), eps=1e-06)
        )
        (mlp): Gemma3MLP(
          (gate_proj): Linear8bitLt(in_features=1152, out_features=6912, bias=False)
          (up_proj): Linear8bitLt(in_features=1152, out_features=6912, bias=False)
          (down_proj): Linear8bitLt(in_features=6912, out_features=1152, bias=False)
          (act_fn): GELUTanh()
        )
        (

In [8]:



def format_dataset(data_point) : 

    input_ = data_point['sentence']
    output_ = data_point['translation']

    # format the data into the required format for the SFT
    MESSAGE_FORMAT = [
    [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a yoda translator."},]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": input_  },]
        },{
            "role": "assistant",
            "content": [{"type": "text", "text": output_},]
        }
    ]]

    # return {"messages" : MESSAGE_FORMAT}

    # encoded = tokenizer.apply_chat_template(
    #     messages, 
    #     tokenize=True, 
    #     add_generation_prompt=False,
    #     return_dict=True,
        
    #     # Truncation and padding might be needed depending on your max_seq_length
    #     truncation=True,
    #     max_length=512 
    # )
    
    # # SFTTrainer expects input_ids and attention_mask
    # return {
    #     "input_ids": encoded["input_ids"],
    #     "attention_mask": encoded["attention_mask"],
    #     "labels": encoded["input_ids"] # Language modeling tasks usually copy inputs to labels
    # }

    text = tokenizer.apply_chat_template(MESSAGE_FORMAT, tokenize=False)
    return {"text" :text[0]} 
    

In [9]:
format_dataset(dataset[13])
# dataset[13]

{'text': '<bos><start_of_turn>user\nYou are a yoda translator.\n\nKick the ball straight and follow through.<end_of_turn>\n<start_of_turn>model\nStraight, kick the ball, and follow through.<end_of_turn>\n'}

In [10]:
dataset = dataset.map(format_dataset,remove_columns=dataset.column_names)


In [11]:
dataset[0]

{'text': '<bos><start_of_turn>user\nYou are a yoda translator.\n\nThe birch canoe slid on the smooth planks.<end_of_turn>\n<start_of_turn>model\nOn the smooth planks, the birch canoe slid.<end_of_turn>\n'}

In [12]:
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id
tokenizer.chat_template



'{{ bos_token }}\n{%- if messages[0][\'role\'] == \'system\' -%}\n    {%- if messages[0][\'content\'] is string -%}\n        {%- set first_user_prefix = messages[0][\'content\'] + \'\n\n\' -%}\n    {%- else -%}\n        {%- set first_user_prefix = messages[0][\'content\'][0][\'text\'] + \'\n\n\' -%}\n    {%- endif -%}\n    {%- set loop_messages = messages[1:] -%}\n{%- else -%}\n    {%- set first_user_prefix = "" -%}\n    {%- set loop_messages = messages -%}\n{%- endif -%}\n{%- for message in loop_messages -%}\n    {%- if (message[\'role\'] == \'user\') != (loop.index0 % 2 == 0) -%}\n        {{ raise_exception("Conversation roles must alternate user/assistant/user/assistant/...") }}\n    {%- endif -%}\n    {%- if (message[\'role\'] == \'assistant\') -%}\n        {%- set role = "model" -%}\n    {%- else -%}\n        {%- set role = message[\'role\'] -%}\n    {%- endif -%}\n    {{ \'<start_of_turn>\' + role + \'\n\' + (first_user_prefix if loop.first else "") }}\n    {%- if message[\'conte

In [13]:

messages = [
    [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a yoda translator."},]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": "The birch canoe slid on the smooth planks."  },]
        }
    ]]


inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(model.device).to(torch.bfloat16)


with torch.inference_mode():
    outputs = model.generate(**inputs, max_new_tokens=64)

outputs = tokenizer.batch_decode(outputs)
print(outputs)

Attempting to cast a BatchEncoding to type torch.bfloat16. This is not supported.
/run/media/srujan/Data/Development/ML-algo-practice/.venv/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


['<bos><start_of_turn>user\nYou are a yoda translator.\n\nThe birch canoe slid on the smooth planks.<end_of_turn>\n<start_of_turn>model\nHmm, yes. Smooth planks, indeed. \n\nLet us see... \n\n*“The birch canoe slid... a gentle dance…”* \n\nIs that understood? <end_of_turn>']


In [14]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)


In [15]:
from trl import SFTTrainer, SFTConfig
from peft import get_peft_model, LoraConfig, TaskType
import transformers

In [16]:

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=SFTConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=1000,
        learning_rate=2e-4,
        bf16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        dataloader_num_workers=2,             # <--- ADDED: Uses 2 CPU threads to load data
        # dataset_kwargs={"skip_prepare_dataset": True} 
    ),
    processing_class=tokenizer,
    peft_config=lora_config,

)

In [ ]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'pad_token_id': 3}.
/run/media/srujan/Data/Development/ML-algo-practice/.venv/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,6.203657
20,2.588155
30,1.779380
40,1.384521
50,1.220133
60,1.085632


In [ ]:
with torch.inference_mode():
    outputs = model.generate(**inputs, max_new_tokens=64)

outputs = tokenizer.batch_decode(outputs)
print(outputs)